In [66]:
from openai import OpenAI
import numpy as np
import pandas as pd

In [67]:
client = OpenAI(
    api_key="sk-K9QGaTdbLnrs4ux3t1zFZQJ41osq8lYQ",
    base_url="https://api.proxyapi.ru/openai/v1",
)

In [68]:
def get_embedding(text):
    model="text-embedding-ada-002"
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [69]:
#find segment text in campaign promt
def find_segment_name_in_promt():

    #read campaign_promt
    with open('campaign_promt.txt', 'r',encoding='utf-8') as file:
    # Read the entire file content into a single string
        content = file.read()
        content = content.replace('\n', ' ')
    promt_to_get_segment_name = '"'+ content + '"' + 'верни только название сегмента'
    chat_completion = client.chat.completions.create(
        model="gpt-4o", messages=[{"role": "system", "content": promt_to_get_segment_name}]
    )
    promt_segment = chat_completion.choices[0].message.content
    return promt_segment


In [70]:
promt_segment = find_segment_name_in_promt()

In [71]:
promt_segment_embedding = get_embedding(promt_segment)

In [84]:
def find_simmilar_segment_num(promt_segment_embedding,segment_embedding_table =  'df_segment_embedding.json'):
    df_segment_embedding = pd.read_json(segment_embedding_table)
    df_segment_embedding['sim_score']= df_segment_embedding.apply(lambda x: np.dot(x['embedding'],promt_segment_embedding), axis=1)
    target_segment = int(df_segment_embedding.iloc[[df_segment_embedding['sim_score'].idxmax()]]['Segment_id'].iloc[0])
    target_segment_sim_score = df_segment_embedding.iloc[[df_segment_embedding['sim_score'].idxmax()]]['sim_score'].iloc[0]
    return target_segment , target_segment_sim_score


In [85]:
target_segment , target_segment_sim_score = find_simmilar_segment_num(promt_segment_embedding)

In [86]:
target_segment

165

In [120]:
target_segment_sim_score

0.9395781808049918

In [129]:
#find segment text in campaign promt
def find_content_name_in_promt():

    #read campaign_promt
    with open('campaign_promt.txt', 'r',encoding='utf-8') as file:
        # Read the entire file content into a single string
        content = file.read()
        content = content.replace('\n', ' ')
    promt_to_get_segment_name = '"'+ content + '"' + 'верни только название шаблона. Важно только название без лишних слов'
    chat_completion = client.chat.completions.create(
        model="gpt-4o", messages=[{"role": "system", "content": promt_to_get_segment_name}]
    )
    promt_content = chat_completion.choices[0].message.content
    return promt_content

In [130]:
promt_content = find_content_name_in_promt()

In [131]:
promt_content

'БонусныеБалл_СМС'

In [132]:
promt_content_embedding = get_embedding(promt_content)

In [133]:
def find_simmilar_content_num(promt_content_embedding,content_embedding_table =  'df_content_embedding.json'):
    df_content_embedding = pd.read_json(content_embedding_table)
    df_content_embedding['sim_score']= df_content_embedding.apply(lambda x: np.dot(x['embedding'],promt_content_embedding), axis=1)
    target_content = int(df_content_embedding.iloc[[df_content_embedding['sim_score'].idxmax()]]['Content_id'].iloc[0])
    target_content_sim_score = df_content_embedding.iloc[[df_content_embedding['sim_score'].idxmax()]]['sim_score'].iloc[0]
    return target_content , target_content_sim_score

In [134]:
target_content , target_content_sim_score = find_simmilar_content_num(promt_content_embedding)


In [135]:
target_content

53

In [136]:
target_content_sim_score

0.9322656492297301